# Predictive model for differential diagnosis

In this notebook, our goal is to develop a model that can take in a patient's symptoms as an input and return a list of the top 3 possible classes (diseases) alongside confidence values for each class expressed as probabilities.


## Library and Data import

**date:** 2021-07-12  

**author:** "Rubanza Silver - Flexible Functions AI Lab"

In [2]:
#|include: false 

%pip install seaborn
%pip install fastkaggle
%pip install -Uqq fastbook
%pip install --upgrade pip
%pip install tqdm
%pip install kagglehub
#%pip install catboost
#%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
#%pip install xgboost
#%pip install lightgbm
#%pip install h2o
#%pip install polars
#%pip install -q -U autogluon.tabular
#%pip install autogluon
#%pip install wandb
#%pip install sweetviz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#| code-fold: true
#| output: false
#| code-summary: "Library Import"

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#import fastbook
#fastbook.setup_book()
#from fastbook import *
from fastai.tabular.all import *
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact
from fastai.imports import *
np.set_printoptions(linewidth=130)
from fastai.text.all import *
from pathlib import Path
import os
import warnings
import gc
import pickle
from joblib import dump, load

# ULMFiT approach

In traditional text transfer learning, We use a pre-trained model called a language model. The model we are also going to use in this example was initially trained on Wikipedia on the task of guessing the next word. We then fine-tuned this model for our disease classification task based on symptoms. We can then use this model for our task of disease classification.

But the Wikipedia English might differ from medical jargon, so to further improve our model. We can employ a technique shown in the [ULMFIT Paper](https://arxiv.org/abs/1801.06146) by Jeremy Howard and Sebastian Ruder. They take the above a step further by fitting the pre-trained model on medical corpus and then using that as a base for our classifier. They noticed that adding this step of training the pretrained model on the task specific corpus gives better result as the model also has better context of the final task.

In [4]:
import kagglehub

my_specific_path = "/data/" 

# Download latest version
path = kagglehub.dataset_download("rubanzasilva/symptoms-disease-no-id"),
output_path=my_specific_path

print("Path to dataset files:", path)

Path to dataset files: ('/teamspace/studios/this_studio/.cache/kagglehub/datasets/rubanzasilva/symptoms-disease-no-id/versions/1',)


In [5]:
path = Path('/teamspace/studios/this_studio/.cache/kagglehub/datasets/rubanzasilva/symptoms-disease-no-id/versions/1')
path

Path('/teamspace/studios/this_studio/.cache/kagglehub/datasets/rubanzasilva/symptoms-disease-no-id/versions/1')

In [6]:
!ls /teamspace/studios/this_studio/.cache/kagglehub/datasets/rubanzasilva/symptoms-disease-no-id/versions/1

models	symptom_disease_no_id_col.csv  symptom_no_id.csv


In [7]:
#symptom_df = pd.read_csv(path_lm/'symptom_synth.csv',index_col=0)
symptom_df = pd.read_csv(path/'symptom_no_id.csv')
sd_df = pd.read_csv(path/'symptom_disease_no_id_col.csv')
symptom_df.head()

,text
0,"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches."
1,"My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation."
2,"I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints."
3,"There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them."
4,"My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms."


In [8]:
symptom_df['text'].nunique(),sd_df['text'].nunique()

(1153, 1153)

## Finetuning a language model with my medical corpus

Below I define a DataLoader which is an extension of PyTorch's DataLoaders class, albeit with more functionality. This takes in our data, and prepares it as input for our model, passing it in batches etc.

The DataLoaders Object allows us to build data objects we can use for training without specifically changing the raw input data.

The dataloader then acts as input for our models. We also pass in valid_pct=0.2 which samples and uses 20% of our data for validation.

In [9]:
#dls_lm = TextDataLoaders.from_df(symptom_df, path=path, is_lm=True, valid_pct=0.2)
dls_lm = TextDataLoaders.from_df(symptom_df, path=path, is_lm=True,text_col='text', valid_pct=0.2)
#dls_lm = TextDataLoaders.from_folder(path=path_lm, is_lm=True, valid_pct=0.1)

We then use show_batch to have a look at some of our data.Since, we are guessing the next word in a sentence, you will notice that the targets have shifted one word to thr right in the *text_* column.

In [10]:
dls_lm.show_batch(max_n=5)

xxbos xxmaj due to a stomach discomfort , i have problems sleeping or staying asleep . i am exhausted after using the bathroom and lose my appetite . xxmaj i 'm quite nervous all the time . xxbos i have a high fever , chills , and severe itching . xxmaj in addition , i have a headache and have been perspiring a lot . xxmaj i 've also been bothered by
have dizziness . xxbos i lose weight xxunk and find it challenging to xxunk weight . i use antacids to xxunk the pain and discomfort i experience . xxmaj my mouth feels horrible . xxbos xxmaj i 've been exhausted and chilly , and i ca n't stop coughing . xxmaj when i do , it pains in my chest and makes my heart xxunk . xxmaj my coughing mucous has a
difficult for me to perform even simple chores . xxbos xxmaj along with xxunk headaches and blurred vision , i suffer acid reflux and trouble xxunk my food . xxbos xxmaj i 've been having trouble going to the restroom recently . xxmaj when i try to go , it 's really uncomfortable , and my anus 

From the above, we notice that the texts were processed and split into tokens. It adds some special tokens like xxbos to indicate the beginning of a text and xxmaj to indicate the next word was capitalised.

We then define a fastai [learner](https://docs.fast.ai/learner.html#learner), which is a fastai class that we can use to handle the training loop. It bundles the essential components needed for training together such as the data, model, the dataloaders, loss functions

We use the AWD LSTM architecture. We are also going to use accuracy and perplexity (the Exponential of the loss) as our metrics for this example. Furthermore, we also set a weight decay (wd) of 0.1 and apply mixed precision (.to_fp16()) to the learner, which speeds up training on GPU'S with tensor cores.


In [11]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

#### Phased Finetuning

A pre-trained model is one that has already been trained on a large dataset and has learnt general patterns and features in a dataset, which can then be used to fine-tune to a specific task. 

By default, the body of the model is frozen, meaning we won’t be updating the parameters of the body during training. For this case, only the head (first few layers) of the model will train.

In [12]:
#| error: false
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


As shown below, we can use the *learn.save* to save the state of our model to a file in learn.path/models/ named “filename.pth”. You can use learn.load('filename') to load the content of this file.

In [13]:
# Now save the model
learn.save('1epoch')

Path('/teamspace/studios/this_studio/.cache/kagglehub/datasets/rubanzasilva/symptoms-disease-no-id/versions/1/models/1epoch.pth')

In [14]:
#| error: false
learn = learn.load('1epoch')

After training the head of the model, we unfreeze the rest of the body and finetune it alongside the head, except for our final layer, which converts activations into probabilities of picking each token in our vocabulary.

In [15]:
#| error: false
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.588778,2.918089,0.403574,18.505892,00:01
1,3.254370,2.531099,0.430845,12.567308,00:01
2,3.032004,2.332018,0.458478,10.298702,00:01
3,2.855797,2.235581,0.484664,9.351917,00:01
4,2.734878,2.223079,0.485388,9.235723,00:01


The model not including the final layers is called an encoder. We use fastai's *save_encoder* to save it as shown below.

In [16]:
#| code-fold: show
#| output: false
#| code-summary: "Save the model"
# Now save the model
learn.save_encoder('finetuned')

Now, that our model has been trained to guess or generate the next word in a sentence, we can use it to create or generate new user inputs that start with the below user input text.

In [17]:
#| output: false
#| error: false
TEXT = "I have running nose, stomach and joint pains"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [18]:
print("\n".join(preds))

i have running nose , stomach and joint pains . There is a lot of mucus in my anus , and some spicy and acidic foods are eaten . My eyes are really red and i have a lot of trouble dealing with it .
i have running nose , stomach and joint pains . My muscles are swollen , and my neck has been really stiff . It 's difficult for me to move without becoming stiff . My joints have been really tight , and my neck has been


## Training a text classifier

We now gather and pass in data to train our text classifier.

In [19]:
#symptom_df = pd.read_csv(path_lm/'symptom_synth.csv',index_col=0)
#sd_df = pd.read_csv(path_lm/'symptom_disease_no_id_col.csv')
sd_df.head()

,label,text
0,Psoriasis,"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches."
1,Psoriasis,"My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation."
2,Psoriasis,"I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints."
3,Psoriasis,"There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them."
4,Psoriasis,"My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms."


In [20]:
# Check for NaN values in the label column
print(sd_df['label'].isna().sum())

# If there are NaNs, you can drop those rows
#df = df.dropna(subset=['label'])

0


In [21]:
#| output: false
#| error: false
#dls_clas = TextDataLoaders.from_df(sd_df, path=path,valid='test', text_vocab=dls_lm.vocab)
dls_clas = TextDataLoaders.from_df(sd_df, path=path,valid='test',text_col='text',label_col='label', text_vocab=dls_lm.vocab)

Passing in *text_vocab=dls_lm.vocab* passes in our previously defined vocabulary to our classifier. 

> To quote the fastai documentation, we have to use the exact same vocabulary as when we were fine-tuning our language model, or the weights learned won’t make any sense.

When you train a language model, it learns to associate specific patterns of numbers (weights) with specific tokens (words or subwords) in your vocabulary. 

Each token is assigned a unique index in the vocabulary, and the model's internal representations (the weights in the embedding layers and beyond) are organised according to these indices.

Think of it like a dictionary where each word has a specific page number. The model learns that information about "good" is on page 382, information about "movie" is on page 1593, and so on. These "page numbers" (indices) must remain consistent for the weights to make sense.

If you were to use a different vocabulary when creating your classifier:
.The token "good" might now be on page 746 instead of 382
.The weights the model learned during language model training were specifically tied to the old index (382)

Now when the classifier sees "good" and looks up page 746, it finds weights that were meant for some completely different word

>This mismatch would render the carefully fine-tuned language model weights essentially random from the perspective of the classifier.

In [22]:
#| error: false
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

We then define our text classifier as shown above. Before training it, we load in the previous encoder.

In [23]:
#| include: false
#from pathlib import Path
#learn.path = Path('/kaggle/working')

In [24]:
#| error: false
learn = learn.load_encoder('finetuned')

#### Discriminative Learning Rates & Gradual Unfreezing

**Discriminative learning** rates means using different learning rates for different layers of the model. 

For example, earlier layers (closer to the input) might get smaller learning rates, while the later layers (closer to the output) get larger learning rates.

**Gradual unfreezing** is a technique where layers of the model are unfrozen (made trainable) incrementally during fine-tuning. 
Instead of unfreezing all layers at once, you start by unfreezing only the topmost layers (closest to the output) and train them first.

Unlike computer vision applications where we unfreeze the model at once, gradual unfreezing has been shown to improve performance for NLP models.




In [25]:
len(dls_lm.vocab)

944

In [26]:
#| error: false
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time


In [27]:
#| error: false
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,1.412581,1.540380,0.745833,00:01


In [28]:
learn.unfreeze()
learn.fit_one_cycle(12, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.036288,1.162449,0.804167,00:02
1,0.979230,0.911138,0.833333,00:02
2,0.906674,0.752576,0.837500,00:02
3,0.825612,0.638481,0.862500,00:02
4,0.739767,0.552024,0.879167,00:02
5,0.661871,0.486698,0.912500,00:02
6,0.591938,0.446377,0.908333,00:02
7,0.523371,0.414159,0.916667,00:02
8,0.472473,0.389775,0.929167,00:02
9,0.429350,0.380760,0.933333,00:02


In [29]:
learn.predict("I am having a running stomach, fever, general body weakness and have been getting bitten by mosquitoes often")

('Hypertension',
 tensor(9),
 tensor([0.0045, 0.0930, 0.0453, 0.1137, 0.0008, 0.0543, 0.0086, 0.0258, 0.0063,
         0.1330, 0.0053, 0.0105, 0.0257, 0.0680, 0.0274, 0.0273, 0.0347, 0.0076,
         0.0815, 0.0816, 0.0481, 0.0384, 0.0382, 0.0207]))

In [30]:
def get_top_3_predictions(text, learn):
    # Get prediction and probabilities
    _, _, probs = learn.predict(text)
    
    # Get the disease labels vocabulary (second list in vocab)
    disease_vocab = learn.dls.vocab[1]  # Access the disease labels
    
    # Get number of classes
    n_classes = len(disease_vocab)
    
    # Get indices of top 3 (or fewer) probabilities
    n_preds = min(3, n_classes)
    top_k_indices = probs.argsort(descending=True)[:n_preds]
    
    # Get the actual labels and their probabilities
    predictions = []
    for idx in top_k_indices:
        label = disease_vocab[int(idx)]
        probability = float(probs[idx])
        predictions.append((label, probability))
    
    return predictions

# Function to format and display the predictions nicely
def display_predictions(predictions):
    for i, (disease, prob) in enumerate(predictions, 1):
        print(f"{i}. {disease}: {prob:.3f}")

In [31]:
test_text = "I am having a running stomach, fever, general body weakness and have been getting bitten by mosquitoes often"
predictions = get_top_3_predictions(test_text, learn)
print("\nTop 3 Predictions:")
display_predictions(predictions)


Top 3 Predictions:
1. Hypertension: 0.133
2. Cervical spondylosis: 0.114
3. Arthritis: 0.093


In [32]:
#| code-fold: true
#| code-summary: "Click to see full code in one cell"
#| error: false
#| echo false

#path = Path('/kaggle/input/symptoms-disease-no-id')
#symptom_df = pd.read_csv(path_lm/'symptom_synth.csv',index_col=0)
symptom_df = pd.read_csv(path/'symptom_no_id.csv')
sd_df = pd.read_csv(path/'symptom_disease_no_id_col.csv')
dls_lm = TextDataLoaders.from_df(symptom_df, path=path,text_col='text', is_lm=True, valid_pct=0.2)
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()
learn.fit_one_cycle(1, 1e-2)
# Create a directory to save the model
#os.makedirs('/kaggle/working/models', exist_ok=True)
# Set the model directory for the learner
#learn.model_dir = '/kaggle/working/models'
# Now save the model
learn.save('1epoch')
learn = learn.load('1epoch')
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)
# Now save the model
learn.save_encoder('finetuned')


#finetuning the classifier
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
dls_clas = TextDataLoaders.from_df(sd_df, path=path,text_col='text',label_col='label', text_vocab=dls_lm.vocab)
#learn.path = Path('/kaggle/working')
learn = learn.load_encoder('finetuned')
learn.fit_one_cycle(1, 2e-2)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))
learn.predict("I am having a running stomach, fever, general body weakness and have been getting bitten by mosquitoes often")

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.335642,3.544831,0.343750,34.633835,00:01


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.596305,2.939593,0.395833,18.908155,00:01
1,3.276862,2.591681,0.430049,13.352204,00:01
2,3.025077,2.394606,0.467593,10.963876,00:01
3,2.855345,2.315545,0.480541,10.130444,00:01
4,2.744909,2.302154,0.485315,9.995688,00:01


epoch,train_loss,valid_loss,accuracy,time
0,2.276270,2.441657,0.445833,00:01


epoch,train_loss,valid_loss,accuracy,time
0,1.438768,1.622265,0.695833,00:01


epoch,train_loss,valid_loss,accuracy,time
0,1.035336,1.109301,0.787500,00:02
1,0.945683,0.887717,0.791667,00:02


('Bronchial Asthma',
 tensor(2),
 tensor([0.0069, 0.0241, 0.2341, 0.0923, 0.0018, 0.0118, 0.0178, 0.0119, 0.0036,
         0.1687, 0.0060, 0.0275, 0.0994, 0.0263, 0.0212, 0.0079, 0.0737, 0.0063,
         0.0091, 0.0648, 0.0063, 0.0283, 0.0252, 0.0249]))

# References

[Fastai Documentation - Text Transfer Learning](https://docs.fast.ai/tutorial.text.html#the-ulmfit-approach)

The dataset for this competition was gotten from [here](https://www.kaggle.com/datasets/niyarrbarman/symptom2disease)

## Next Steps

* Using clinical guidelines as a medical corpus source.

* Implementing a newer architecture, e.g., replacing AWD_LSTM with transformers.

* Try out a RAG implementation 

* Finetune our own medical model

* Adding reasoning

* Building a UI
